# Co-Simulation

In this example, we simulate a basic energy system of a small datacenter equipped with on-site solar power and a simple battery.

In [1]:
import pandas as pd

from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment
from vessim.power_meter import MockPowerMeter
from vessim.signal import SAMSignal
from vessim.storage import SimpleBattery

# Hotfix to execute asyncio in Jupyter
import nest_asyncio
nest_asyncio.apply()

We define an `Environment` with a single `Microgird` that comprises

- A simulated `ComputingSystem` with two servers that consistently draw 200W and 250W, respectively. The datacenter has a [power usage effectiveness](https://en.wikipedia.org/wiki/Power_usage_effectiveness) of 1.6, resulting in a total power demand of 1.6 * (200 W + 250 W) = 720 W.
- A `Generator` which represents a wind turbine, modeled according to the dataset `solcast2022_global` and the configuration file `windpower_config`.
- A `SimpleBattery` with a usable capacity of 100 Wh which is charged with 1Wh.
- A `Monitor` which periodically stores the microgrid state and eventually writes it to a CSV file.

In [2]:
environment = Environment(sim_start="2020-06-11 00:00:00")

monitor = Monitor()  # stores simulation result on each step
environment.add_microgrid(
    actors=[
        ComputingSystem(power_meters=[MockPowerMeter(p=200), MockPowerMeter(p=250)], pue=1.6),
        Generator(signal=SAMSignal(model="Windpower", weather_file="./data/WIND-Toolkit_lat44.32_lon-74.13_2014_5min_flat_lands_northern_ny.csv", config_file="./data/windpower_config.json"))
    ],
    controllers=[monitor],
    storage=SimpleBattery(capacity=100, charge_level=100),
    step_size=60,  # global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600)  # 24h
monitor.to_csv("result.csv")